<a href="https://colab.research.google.com/github/LuViBeBe93/tesis/blob/main/GPT2_F4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers pandas torch

In [2]:
from google.colab import drive
import pandas as pd
from google.colab import drive
import shutil
from google.colab import drive
#import nltk

#Aquí seleccionamos el archivo desde nuestro equipo.

from google.colab import files
uploaded = files.upload()

Saving BASE_30122024.xlsx to BASE_30122024.xlsx


In [3]:
import pandas as pd

In [4]:
data = pd.read_excel('BASE_30122024.xlsx')

In [5]:
data = pd.DataFrame(data)

In [6]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, AutoTokenizer, AutoModelWithLMHead

# Configuración para usar GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Cargar los datos económicos
data

# Convertir datos numéricos a texto
def datos_a_texto(row):
    texto_datos = (
        f"Año: {row.get('Año', 'NaN')}, Mes: {row.get('Mes', 'NaN')}, "
        f"Trimestre: {row.get('Trimestre', 'NaN')}, Frecuencia: {row.get('Frecuencia', 'NaN')}, "
        f"Año_comparación: {row.get('Año_comparación', 'NaN')}, País: {row.get('País', 'NaN')}, "
        f"Tendencia: {row.get('Tendencia', 'NaN')}, Sector: {row.get('Sector', 'NaN')}, "
        f"Indicador: {row.get('Indicador', 'NaN')}, Valor_actual: {row.get('Valor_actual', 'NaN')}, "
        f"Variación: {row.get('Variación', 'NaN')}, Valor_comparación: {row.get('Valor_comparación', 'NaN')}, "
        f"Entidad: {row.get('Entidad', 'NaN')}"
    )
    return texto_datos

# Crear una columna con los datos numéricos convertidos a texto
data["entrada_texto"] = data.apply(datos_a_texto, axis=1)
data["texto_final"] = data["entrada_texto"] + " </s> Noticia: " + data["Noticia"]


In [7]:
# Dataset personalizado
class NoticiasDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        texto = self.data.iloc[idx]["texto_final"]
        encoding = self.tokenizer(
            texto,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt",
        )
        input_ids = encoding["input_ids"].flatten()
        attention_mask = encoding["attention_mask"].flatten()

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": input_ids,  # En GPT, las etiquetas son las mismas que la entrada
        }

In [8]:
# Cargar el tokenizador y modelo GPT-2 preentrenado
tokenizer = AutoTokenizer.from_pretrained("datificate/gpt2-small-spanish")
model = AutoModelWithLMHead.from_pretrained("datificate/gpt2-small-spanish")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/817 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/850k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/508k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1838: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

In [9]:
# Ajustar el tokenizador para manejar padding
tokenizer.pad_token = tokenizer.eos_token
model.resize_token_embeddings(len(tokenizer))

# Crear el dataset y el DataLoader
dataset = NoticiasDataset(data, tokenizer)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

# Configuración del entrenamiento
training_args = TrainingArguments(
    output_dir="./gpt2_noticias_model",
    num_train_epochs=7, #Este parámetro sale de la optimización bayesiana con Optuna
    per_device_train_batch_size=4, #Este parámetro sale de la optimización bayesiana con Optuna
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=100,
)

# Crear el Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

# Entrenar el modelo
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
100,1.301000
200,0.495400
300,0.417000
400,0.367900
500,0.349700
600,0.313400
700,0.305200
800,0.301300
900,0.275600
1000,0.269800


Step,Training Loss
100,1.301000
200,0.495400
300,0.417000
400,0.367900
500,0.349700
600,0.313400
700,0.305200
800,0.301300
900,0.275600
1000,0.269800


TrainOutput(global_step=5579, training_loss=0.2480914850434985, metrics={'train_runtime': 1145.7735, 'train_samples_per_second': 19.477, 'train_steps_per_second': 4.869, 'total_flos': 1457748246528000.0, 'train_loss': 0.2480914850434985, 'epoch': 7.0})

In [10]:
def generar_noticia(model, tokenizer, datos):
    entrada_texto = datos_a_texto(datos)
    inputs = tokenizer.encode(entrada_texto, return_tensors="pt").to(model.device)

    # Asegurar que el modelo tenga `pad_token_id` configurado correctamente
    if model.config.pad_token_id is None:
        model.config.pad_token_id = tokenizer.eos_token_id

    # Generar texto
    outputs = model.generate(
        inputs,
        max_length=128,
        num_beams=3, #Este parámetro sale de la optimización bayesiana con Optuna
        no_repeat_ngram_size=3, #Este parámetro sale de la optimización bayesiana con Optuna
        early_stopping=True,
        attention_mask=torch.ones_like(inputs),  # Crear un attention_mask válido
        temperature=0.7,  #Este parámetro sale de la optimización por grilla
        top_p=0.9,  #Este parámetro sale de la optimización por grilla
        do_sample=True  # Activa la aleatorización (en lugar de beam search)
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)



In [11]:
# Ejemplo 1
new_data = {
    'Año': '2024',
    'Mes': 'enero',
    'Trimestre': '',
    'Frecuencia': 'Mensual',
    'Año_comparación': '2023',
    'País': 'Colombia',
    'Tendencia': 'Crecimiento',
    'Sector': 'Economía',
    'Indicador': 'Inflación',
    'Valor_actual': '9%',
    'Variación': '1%',
    'Valor_comparación': '8%',
    'Entidad': 'DANE',
}

noticia = generar_noticia(model, tokenizer, new_data)
print("Noticia generada:", noticia)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Noticia generada: Año: 2024, Mes: enero, Trimestre: , Frecuencia: Mensual, Año_comparación: 2023, País: Colombia, Tendencia: Crecimiento, Sector: Economía, Indicador: Inflación, Valor_actual: 9%, Variación: 1%, Valor_comparación: 8%, Entidad: DANE </s> Noticia: En Colombia, el Dane informó que en enero de 2024 la variación mensual de la inflación fue de 9.0% en su medición anual..0 millones de dólares de inversión extranjera directa (IED) en Colombia


In [12]:
# Ejemplo 2
new_data = {
    'Año': '2024',
    'Mes': '',
    'Trimestre': 'Primer',
    'Frecuencia': 'Trimestral',
    'Año_comparación': '',
    'País': 'Argentina',
    'Tendencia': 'Crecimiento',
    'Sector': 'Economía',
    'Indicador': 'PIB',
    'Valor_actual': '10%',
    'Variación': '',
    'Valor_comparación': '',
    'Entidad': '',
}

noticia = generar_noticia(model, tokenizer, new_data)
print("Noticia generada:", noticia)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Noticia generada: Año: 2024, Mes: , Trimestre: Primer, Frecuencia: Trimestral, Año_comparación: , País: Argentina, Tendencia: Crecimiento, Sector: Economía, Indicador: PIB, Valor_actual: 10%, Variación: , Valor_comparación: , Entidad: </s> Noticia: La economía de Argentina creció un 10% en el primer trimestre de 2024 en comparación con el mismo período del año anterior., Banco Mundial _ millones de dólares en un primer trimestre, un crecimiento del 10% respecto al mismo


In [13]:
# Ejemplo 3
new_data = {
    'Año': '2020',
    'Mes': 'mayo',
    'Trimestre': '',
    'Frecuencia': 'Mensual',
    'Año_comparación': '',
    'País': 'Bolivia',
    'Tendencia': 'Disminuye',
    'Sector': 'Economía',
    'Indicador': 'Tasa_desempleo',
    'Valor_actual': '3%',
    'Variación': '',
    'Valor_comparación': '',
    'Entidad': '',
}

noticia = generar_noticia(model, tokenizer, new_data)
print("Noticia generada:", noticia)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Noticia generada: Año: 2020, Mes: mayo, Trimestre: , Frecuencia: Mensual, Año_comparación: , País: Bolivia, Tendencia: Disminuye, Sector: Economía, Indicador: Tasa_desempleo, Valor_actual: 3%, Variación: , Valor_comparación: , Entidad: </s> Noticia: La tasa de desempleo en Bolivia descendió al 3% en mayo de 2020 sempleo: 0.096, Variaciones: nan, Valor: nan Farnes en su medición anual se ubicó en 9.6


In [19]:
# Ejemplo 4
new_data = {
    'Año': '1993',
    'Mes': '',
    'Trimestre': '',
    'Frecuencia': 'Anual',
    'Año_comparación': '1992',
    'País': 'Nicaragua',
    'Tendencia': 'Disminuye',
    'Sector': 'Economía',
    'Indicador': 'Importaciones',
    'Valor_actual': '12000',
    'Variación': '1%',
    'Valor_comparación': '',
    'Entidad': '',
}

noticia = generar_noticia(model, tokenizer, new_data)
print("Noticia generada:", noticia)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Noticia generada: Año: 1993, Mes: , Trimestre: , Frecuencia: Anual, Año_comparación: 1992, País: Nicaragua, Tendencia: Disminuye, Sector: Economía, Indicador: Importaciones, Valor_actual: 12000, Variación: 1%, Valor_comparación: , Entidad: </s> Noticia: Las importaciones en Nicaragua cayeron un 1% en 1993 en comparación con el mismo periodo de 1992 se ubicaron en los 12000 millones de dólares se ubicó en los sectores de Economía y comercio exterior del país.> Tasa


In [22]:
# Ejemplo 5
new_data = {
    'Año': '1993',
    'Mes': '',
    'Trimestre': '',
    'Frecuencia': 'Anual',
    'Año_comparación': '1992',
    'País': 'Uruguay',
    'Tendencia': 'Disminuye',
    'Sector': 'Economía',
    'Indicador': 'Exportaciones',
    'Valor_actual': '10000',
    'Variación': '1%',
    'Valor_comparación': '',
    'Entidad': '',
}

noticia = generar_noticia(model, tokenizer, new_data)
print("Noticia generada:", noticia)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Noticia generada: Año: 1993, Mes: , Trimestre: , Frecuencia: Anual, Año_comparación: 1992, País: Uruguay, Tendencia: Disminuye, Sector: Economía, Indicador: Exportaciones, Valor_actual: 10000, Variación: 1%, Valor_comparación: , Entidad: </s> Noticia: Las exportaciones en Uruguay cayeron un 10% en 1993, alcanzando un valor de 10000 dólares por barril. en Uruguay se recuperará en 1993 con un aumento proyectado del 11.5%. se ubicó en 10.5%


In [24]:
# Ejemplo 6
new_data = {
    'Año': '2015',
    'Mes': 'noviembre',
    'Trimestre': '',
    'Frecuencia': 'Mensual',
    'Año_comparación': '',
    'País': 'Chile',
    'Tendencia': 'Disminuye',
    'Sector': 'Economía',
    'Indicador': 'Precio_petróleo',
    'Valor_actual': '81',
    'Variación': '',
    'Valor_comparación': '',
    'Entidad': '',
}

noticia = generar_noticia(model, tokenizer, new_data)
print("Noticia generada:", noticia)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Noticia generada: Año: 2015, Mes: noviembre, Trimestre: , Frecuencia: Mensual, Año_comparación: , País: Chile, Tendencia: Disminuye, Sector: Economía, Indicador: Precio_petróleo, Valor_actual: 81, Variación: , Valor_comparación: , Entidad: </s> Noticia: En noviembre de 2015, el precio del petróleo en Chile muestra una tendencia hacia la baja, en parte debido a factores como la menor demanda global y la transición hacia energías renovables, que afecta el mercado de combustibles fósiles en Chile durante el mes.


In [25]:
# Ejemplo 7
new_data = {
    'Año': '2015',
    'Mes': 'noviembre',
    'Trimestre': '',
    'Frecuencia': 'Mensual',
    'Año_comparación': '',
    'País': 'Italia',
    'Tendencia': 'Disminuye',
    'Sector': 'Economía',
    'Indicador': 'Inversión_extranjera_directa',
    'Valor_actual': '19000',
    'Variación': '',
    'Valor_comparación': '',
    'Entidad': '',
}

noticia = generar_noticia(model, tokenizer, new_data)
print("Noticia generada:", noticia)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Noticia generada: Año: 2015, Mes: noviembre, Trimestre: , Frecuencia: Mensual, Año_comparación: , País: Italia, Tendencia: Disminuye, Sector: Economía, Indicador: Inversión_extranjera_directa, Valor_actual: 19000, Variación: , Valor_comparación: , Entidad: </s> Noticia: La IED en Italia alcanzó los 19000 millones de dólares en noviembre de 2015, lo que representa un crecimiento del 10.7% en comparación con el mismo mes del año anterior. 0.4%
